In [ ]:
import tensorflow as tf
import numpy as np
import os, zipfile
from glob import glob

# Custom InstanceNormalization (no tensorflow_addons)
class InstanceNormalization(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-5):
        super().__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale', shape=input_shape[-1:], initializer='ones', trainable=True)
        self.offset = self.add_weight(name='offset', shape=input_shape[-1:], initializer='zeros', trainable=True)

    def call(self, x):
        mean, var = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        return self.scale * (x - mean) / tf.sqrt(var + self.epsilon) + self.offset

In [ ]:
MONET_DIR = '/kaggle/input/gan-getting-started/monet_jpg'
PHOTO_DIR = '/kaggle/input/gan-getting-started/photo_jpg'

In [ ]:
IMG_SIZE = 256

def preprocess(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = (tf.cast(img, tf.float32) / 127.5) - 1
    return img

photo_paths = glob(f'{PHOTO_DIR}/*.jpg')
photo_ds = tf.data.Dataset.from_tensor_slices(photo_paths).map(preprocess).batch(1)

In [ ]:
def downsample(filters, size, norm=True):
    init = tf.random_normal_initializer(0., 0.02)
    block = tf.keras.Sequential()
    block.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=init, use_bias=False))
    if norm:
        block.add(InstanceNormalization())
    block.add(tf.keras.layers.LeakyReLU())
    return block

def upsample(filters, size):
    init = tf.random_normal_initializer(0., 0.02)
    block = tf.keras.Sequential()
    block.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer=init, use_bias=False))
    block.add(InstanceNormalization())
    block.add(tf.keras.layers.ReLU())
    return block

In [ ]:
def build_generator():
    inputs = tf.keras.layers.Input(shape=[IMG_SIZE, IMG_SIZE, 3])
    x = inputs
    for down in [downsample(64, 4, False), downsample(128, 4), downsample(256, 4)]:
        x = down(x)
    for up in [upsample(128, 4), upsample(64, 4)]:
        x = up(x)
    x = tf.keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(x)
    return tf.keras.Model(inputs=inputs, outputs=x)

generator_g = build_generator()  # Photo → Monet

In [ ]:
def generate(model, input_img):
    pred = model(input_img, training=False)
    pred = (pred + 1) * 127.5
    return tf.cast(pred, tf.uint8)

os.makedirs('generated_images', exist_ok=True)

for i, img in enumerate(photo_ds.take(7000)):
    out = generate(generator_g, img)
    encoded = tf.io.encode_jpeg(tf.squeeze(out))
    tf.io.write_file(f'generated_images/{i:05d}.jpg', encoded)

In [ ]:
with zipfile.ZipFile('images.zip', 'w') as zipf:
    for path in glob('generated_images/*.jpg'):
        zipf.write(path, arcname=os.path.basename(path))

In [ ]:
print("Zip exists:", os.path.exists("images.zip"))
print("Image count:", len(glob("generated_images/*.jpg")))